<a href="https://colab.research.google.com/github/EmilSeyfullayev/Credit_Risk_Modeling_in_Python/blob/main/Preprocess_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from datetime import timedelta

df = pd.read_csv("/content/drive/MyDrive/Credit Risk Modeling/loan_data_2007_2014.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Employment Length

In [172]:
df['emp_length'] = df['emp_length'].str.replace('< 1 year', str(0))
df['emp_length'].fillna(str(0), inplace=True)
df['emp_length'] = df['emp_length'].str.extract('(\d+)')[0]
df['emp_length'] = pd.to_numeric(df['emp_length'])

### Earliest Credit line

In [173]:
earliest_credit_line_1 = pd.to_datetime(df['earliest_cr_line'], format = '%b-%y')
hundred_years = 75*365 + 25*366
df['earliest_credit_line_1'] = earliest_credit_line_1
df['earliest_credit_line_2'] = df['earliest_credit_line_1'].apply(lambda x : 
                                   x.date() - timedelta(days = hundred_years) 
                                   if x > np.datetime64('2015-01-01')
                                   else x)
# earliest_credit_line_3 is the variable shows time difference in months between 2022-01-01 and earliest_credit_line
df['earliest_credit_line_3'] = df['earliest_credit_line_2'].apply(
    lambda x: (
        (np.datetime64('2022-01-01')-x)/30
    ).days
  )

### Months since issue date

In [174]:
df['issue_d_date'] = pd.to_datetime(df['issue_d'], format = '%b-%y')

df['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - 
                                                       df['issue_d_date']) / np.timedelta64(1, 'M')))


### Term

In [175]:
term_df = pd.get_dummies(df['term'], prefix = 'term', prefix_sep = ':', 
               #drop_first=True
               )
df = pd.concat([df, term_df], axis=1)

### Issue date

In [176]:
df['issue_date_1'] = pd.to_datetime(df['issue_d'], format = '%b-%y')
df['issue_date_2'] = df['issue_date_1'].apply(
    lambda x: round(np.float((
        (np.datetime64('2022-01-01') - x).days
    )/30))
)

### Some discrete variables

In [177]:
dummies = [pd.get_dummies(df['grade'],              prefix = 'grade',               prefix_sep = ':'),
          pd.get_dummies(df['sub_grade'],           prefix = 'sub_grade',           prefix_sep = ':'),
          pd.get_dummies(df['home_ownership'],      prefix = 'home_ownership',      prefix_sep = ':'),
          pd.get_dummies(df['verification_status'], prefix = 'verification_status', prefix_sep = ':'),
          pd.get_dummies(df['loan_status'],         prefix = 'loan_status',         prefix_sep = ':'),
          pd.get_dummies(df['purpose'],             prefix = 'purpose',             prefix_sep = ':'),
          pd.get_dummies(df['addr_state'],          prefix = 'addr_state',          prefix_sep = ':'),
          pd.get_dummies(df['initial_list_status'], prefix = 'initial_list_status', prefix_sep = ':')]

dummies = pd.concat(dummies, axis=1)
df = pd.concat([df, dummies], axis=1)

### Variables which have missing and we need them

In [178]:
df['total_rev_hi_lim'].fillna(df['funded_amnt'], inplace=True)
df['annual_inc'].fillna(df['annual_inc'].mean(), inplace=True)

df['earliest_credit_line_2'].fillna(0, inplace=True)
df['earliest_credit_line_3'].fillna(0, inplace=True)
df['acc_now_delinq'].fillna(0, inplace=True)
df['total_acc'].fillna(0, inplace=True)
df['pub_rec'].fillna(0, inplace=True)
df['open_acc'].fillna(0, inplace=True)
df['inq_last_6mths'].fillna(0, inplace=True)
df['delinq_2yrs'].fillna(0, inplace=True)
df['emp_length'].fillna(0, inplace=True)

### Dependent variable and train test split

In [179]:
defaults = [
            'Charged Off',
            'Default',
            'Late (16-30 days)',
            'Does not meet the credit policy. Status:Charged Off'
]
good_loan = np.where(
    df['loan_status'].isin(defaults),
    0, 1
)
df['good_loan'] = good_loan

X = df.drop('good_loan', axis=1)
y = df['good_loan']


In [180]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# df = pd.concat([X_train, y_train], axis=1)
df = pd.concat([X_test, y_test], axis=1)

### Weight of Evidence function used for preprocessing

In [181]:
def WoE_and_IV(df, variable_name):

  bad = np.where(df['good_loan']== 1, 0, 1)
  good = df['good_loan']

  table = pd.DataFrame()
  table[variable_name] = df[variable_name]
  table['good_count'] = good
  table['bad_count'] = bad
  table['numb_of_obs'] = table['good_count'] + table['bad_count']

  table = table.groupby(table.columns[0], as_index=False).sum()

  table['proportion_of_good'] = table['good_count']/table['good_count'].sum()
  table['proportion_of_bad']  = table['bad_count'] /table['bad_count'].sum()

  table['Weight_of_Evidence'] = np.log(
      table['proportion_of_good']/table['proportion_of_bad']
  )

  table['Information_value'] = (
      table['proportion_of_good'] - table['proportion_of_bad']
  ) * table['Weight_of_Evidence']

  table['Information_value'] = table['Information_value'].sum()

  table = table.sort_values('Weight_of_Evidence', ascending = True)
  table = table.reset_index(drop=True)

  return table

def vis_WoE(table, rotation=0, width=8, height=4):
  variable_name = table.columns.values[0]
  x = table[variable_name].apply(str)
  y = table['Weight_of_Evidence']
  numb_of_obs = table['numb_of_obs']

  # g = sns.FacetGrid(table)
  # g.map(sns.scatterplot(variable_name, y))
  # g.map(plt.lineplot(variable_name, y))
  plt.figure(figsize=(width,height), dpi=180)
  plt.xticks(rotation=rotation)
  plt.legend(markerscale = 10)
  sns.scatterplot(x = x, y = y, 
                  # data = table,
               # marker = 'o', 
               # linestyle='--', 
               size = numb_of_obs, 
               # s = 100,
               hue = numb_of_obs,
               # palette = 'Blues',
               edgecolor = 'red'
           )

### Home ownership

In [182]:
df['home_ownership:ANY_OTHER_NONE_RENT'] = sum([
    df['home_ownership:ANY'], df['home_ownership:NONE'],
    df['home_ownership:OTHER'], df['home_ownership:RENT']
])

### States

In [183]:
class States():

  def __init__(self, got_column_names="None"):
    self.got_column_names = got_column_names
    

  def states_binning(self, df):

    states_groups = [
                    ['ND','NE', 'IA', 'NV', 'AL', 'NM'],
                    ['FL'],
                    ['HI', 'MO', 'VA'],
                    ['CA'],
                    ['OK', 'TN', 'NJ', 'NC', 'KY'],
                    ['NY'],
                    ['MD', 'UT', 'LA', 'MN', 'MI'],
                    ['SD', 'WA', 'AZ', 'OH', 'PA', 'MA', 'AR', 'DE', 'IN', 'GA'],
                    ['WI', 'RI', 'OR'],
                    ['CT', 'IL'],
                    ['TX'],
                    ['MT', 'KS', 'CO', 'SC', 'AK'],
                    ['VT', 'MS', 'NH', 'WV', 'WY', 'DC', 'ME', 'ID']
    ]

    self.got_column_names = []

    for group in states_groups:

      group_column_names = ['addr_state:{}'.format(state) for state in group]

      existing_column_names = []

      for column_name in group_column_names:
        if column_name in df.columns.values:
          existing_column_names.append(column_name)
      
      existing_states_XX_of_group = []

      for existing_column_name in existing_column_names:
        try:
          two_parts_of_column_name = existing_column_name.split(":")
          existing_states_XX_of_group.append(two_parts_of_column_name[1])
        except IndexError:
          pass 

      df["addr_state:" + "_".join(existing_states_XX_of_group)] =  df[['addr_state:{}'.format(state) for state in existing_states_XX_of_group]].sum(axis=1)
      
      self.got_column_names.append(
          "addr_state:" + "_".join(existing_states_XX_of_group)
      )


  def got_column_names_func(self):
    return self.got_column_names

state = States()
state.states_binning(df)

### Purpose of the loan

In [184]:
df['purpose:educ__sm_b__wedd__ren_en__mov__house'] = sum([df['purpose:educational'], 
                                                          df['purpose:small_business'],
                                                          df['purpose:wedding'],
                                                          df['purpose:renewable_energy'],
                                                          df['purpose:moving'],
                                                          df['purpose:house']])

df['purpose:oth__med__vacation'] = sum([df['purpose:other'], 
                                        df['purpose:medical'],
                                        df['purpose:vacation']])

df['purpose:major_purch__car__home_impr'] = sum([df['purpose:major_purchase'], 
                                                 df['purpose:car'],
                                                  df['purpose:home_improvement']])

### Employment Length

In [185]:
def emp_legth_WoE_binner(data, column_name = 'emp_length'):

  table = pd.DataFrame()

  table['emp_length:0'] = np.where(data[column_name].isin([0]), 1, 0)
  table['emp_length:1'] = np.where(data[column_name].isin([1]), 1, 0)
  table['emp_length:2-4'] = np.where(data[column_name].isin(range(2, 5)), 1, 0)
  table['emp_length:5-6'] = np.where(data[column_name].isin(range(5, 7)), 1, 0)
  table['emp_length:7-9'] = np.where(data[column_name].isin(range(7, 10)), 1, 0)
  table['emp_length:10'] = np.where(data[column_name].isin([10]), 1, 0)

  table = pd.concat([data, table], axis=1)

  return table

temp = emp_legth_WoE_binner(df)
df = temp
del(temp)

### Months since issue date

In [186]:
df['since_issue_date:<87']  = np.where(df['issue_date_2'].isin(range(88)),      1, 0)
df['since_issue_date:<90']  = np.where(df['issue_date_2'].isin(range(88,  90)), 1, 0)
df['since_issue_date:<92']  = np.where(df['issue_date_2'].isin(range(90,  92)), 1, 0)
df['since_issue_date:<94']  = np.where(df['issue_date_2'].isin(range(92,  94)), 1, 0)
df['since_issue_date:<99']  = np.where(df['issue_date_2'].isin(range(94,  99)), 1, 0)
df['since_issue_date:<103'] = np.where(df['issue_date_2'].isin(range(99, 103)), 1, 0)
df['since_issue_date:<112'] = np.where(df['issue_date_2'].isin(range(103,112)), 1, 0)
df['since_issue_date:<142'] = np.where(df['issue_date_2'].isin(range(112,142)), 1, 0)
df['since_issue_date:>=142'] = np.where(df['issue_date_2']>= 142, 1, 0)

### Interest Rate

In [187]:
df['interest_rate:<9.548']        = np.where(df['int_rate']<=9.548,                              1, 0)
df['interest_rate:9.548-12.025']  = np.where((df['int_rate']>9.548)  & (df['int_rate']<=12.025), 1, 0)
df['interest_rate:12.025-14.089'] = np.where((df['int_rate']>12.025) & (df['int_rate']<=14.089), 1, 0)
df['interest_rate:14.089-15.74']  = np.where((df['int_rate']>14.089) & (df['int_rate']<=15.74),  1, 0)
df['interest_rate:15.74-20.281']  = np.where((df['int_rate']>15.74)  & (df['int_rate']<=20.281), 1, 0)
df['interest_rate:>20.281']       = np.where((df['int_rate']>20.281),                            1, 0)

### Earliest credit line

In [188]:
df['earl_cr_line:<209']    = np.where( df['earliest_credit_line_3']<209, 1, 0)
df['earl_cr_line:209-285'] = np.where((df['earliest_credit_line_3']>=209) & (df['earliest_credit_line_3']<285), 1, 0)
df['earl_cr_line:285-323'] = np.where((df['earliest_credit_line_3']>=285) & (df['earliest_credit_line_3']<323), 1, 0)
df['earl_cr_line:323-380'] = np.where((df['earliest_credit_line_3']>=323) & (df['earliest_credit_line_3']<380), 1, 0)
df['earl_cr_line:380-437'] = np.where((df['earliest_credit_line_3']>=380) & (df['earliest_credit_line_3']<437), 1, 0)
df['earl_cr_line:>=437']   = np.where( df['earliest_credit_line_3']>=437, 1, 0)

### Delinquency

In [189]:
df['delinq_2yrs:0'] = np.where((df['delinq_2yrs'] == 0), 1, 0)
df['delinq_2yrs:1-3'] = np.where((df['delinq_2yrs'] >= 1) & (df['delinq_2yrs'] <= 3), 1, 0)
df['delinq_2yrs:4-11'] = np.where((df['delinq_2yrs'] >= 4) & (df['delinq_2yrs'] <= 11), 1, 0)
df['delinq_2yrs:>=12'] = np.where((df['delinq_2yrs'] >= 12), 1, 0)

### Inquiry last 6 months

In [190]:
df['inq_last_6mths:0'] = np.where((df['inq_last_6mths'] == 0), 1, 0)
df['inq_last_6mths:1-2'] = np.where((df['inq_last_6mths'] >= 1) & (df['inq_last_6mths'] <= 2), 1, 0)
df['inq_last_6mths:3-6'] = np.where((df['inq_last_6mths'] >= 3) & (df['inq_last_6mths'] <= 6), 1, 0)
df['inq_last_6mths:>6'] = np.where((df['inq_last_6mths'] > 6), 1, 0)

### Open accounts

In [191]:
df['open_acc:0']     = np.where((df['open_acc'] == 0), 1, 0)
df['open_acc:1-3']   = np.where((df['open_acc'] >= 1)  & (df['open_acc'] <= 3), 1, 0)
df['open_acc:4-9']   = np.where((df['open_acc'] >= 4)  & (df['open_acc'] <= 9), 1, 0)
df['open_acc:10-12'] = np.where((df['open_acc'] >= 11) & (df['open_acc'] <= 12), 1, 0)
df['open_acc:13-17'] = np.where((df['open_acc'] >= 13) & (df['open_acc'] <= 17), 1, 0)
df['open_acc:18-22'] = np.where((df['open_acc'] >= 18) & (df['open_acc'] <= 22), 1, 0)
df['open_acc:23-27'] = np.where((df['open_acc'] >= 23) & (df['open_acc'] <= 27), 1, 0)
df['open_acc:28-35'] = np.where((df['open_acc'] >= 26) & (df['open_acc'] <= 35), 1, 0)
df['open_acc:>=36']  = np.where((df['open_acc'] >= 31), 1, 0) 

### Public records

In [192]:
df['pub_rec:0'] = np.where((df['pub_rec'] == 0), 1, 0)
df['pub_rec:1-2'] = np.where((df['pub_rec'] >= 1) & (df['pub_rec'] <= 2), 1, 0)
df['pub_rec:3-5'] = np.where((df['pub_rec'] >= 3) & (df['pub_rec'] <= 5), 1, 0)
df['pub_rec:>=6'] = np.where((df['pub_rec'] >= 6), 1, 0)

### Total accounts

In [193]:
df['total_acc:<=3']    = np.where((df['total_acc'] <= 3), 1, 0)
df['total_acc:4-15']   = np.where((df['total_acc'] >= 4)  & (df['total_acc'] <= 15), 1, 0)
df['total_acc:16-21']  = np.where((df['total_acc'] >= 16) & (df['total_acc'] <= 21), 1, 0)
df['total_acc:22-27']  = np.where((df['total_acc'] >= 22) & (df['total_acc'] <= 27), 1, 0)
df['total_acc:28-48']  = np.where((df['total_acc'] >= 28) & (df['total_acc'] <= 48), 1, 0)
df['total_acc:49-63']  = np.where((df['total_acc'] >= 49) & (df['total_acc'] <= 63), 1, 0)
df['total_acc:64-78']  = np.where((df['total_acc'] >= 64) & (df['total_acc'] <= 78), 1, 0)
df['total_acc:>=79']   = np.where((df['total_acc'] >= 79), 1, 0)

### Accounts now delinquent

In [194]:
df['acc_now_delinq:0'] = np.where((df['acc_now_delinq'] == 0), 1, 0)
df['acc_now_delinq:>=1'] = np.where((df['acc_now_delinq'] >= 1), 1, 0) # number of observations in next ones is very low

### Total revolving high limit

In [195]:
df['total_rev_hi_lim:<=5K'] = np.where((df['total_rev_hi_lim'] <= 5000), 1, 0)
df['total_rev_hi_lim:5K-10K'] = np.where((df['total_rev_hi_lim'] > 5000) & (df['total_rev_hi_lim'] <= 10000), 1, 0)
df['total_rev_hi_lim:10K-15K'] = np.where((df['total_rev_hi_lim'] > 10000) & (df['total_rev_hi_lim'] <= 15000), 1, 0)
df['total_rev_hi_lim:15K-20K'] = np.where((df['total_rev_hi_lim'] > 15000) & (df['total_rev_hi_lim'] <= 20000), 1, 0)
df['total_rev_hi_lim:20K-30K'] = np.where((df['total_rev_hi_lim'] > 20000) & (df['total_rev_hi_lim'] <= 30000), 1, 0)
df['total_rev_hi_lim:30K-40K'] = np.where((df['total_rev_hi_lim'] > 30000) & (df['total_rev_hi_lim'] <= 40000), 1, 0)
df['total_rev_hi_lim:40K-55K'] = np.where((df['total_rev_hi_lim'] > 40000) & (df['total_rev_hi_lim'] <= 55000), 1, 0)
df['total_rev_hi_lim:55K-95K'] = np.where((df['total_rev_hi_lim'] > 55000) & (df['total_rev_hi_lim'] <= 95000), 1, 0)
df['total_rev_hi_lim:>95K'] = np.where((df['total_rev_hi_lim'] > 95000), 1, 0)

### Annual income

In [196]:
df['annual_inc:<20K']      = np.where((df['annual_inc'] <= 20000), 1, 0)
df['annual_inc:20K-30K']   = np.where((df['annual_inc'] > 20000)  & (df['annual_inc'] <= 30000), 1, 0)
df['annual_inc:30K-40K']   = np.where((df['annual_inc'] > 30000)  & (df['annual_inc'] <= 40000), 1, 0)
df['annual_inc:40K-50K']   = np.where((df['annual_inc'] > 40000)  & (df['annual_inc'] <= 50000), 1, 0)
df['annual_inc:50K-60K']   = np.where((df['annual_inc'] > 50000)  & (df['annual_inc'] <= 60000), 1, 0)
df['annual_inc:60K-70K']   = np.where((df['annual_inc'] > 60000)  & (df['annual_inc'] <= 70000), 1, 0)
df['annual_inc:70K-80K']   = np.where((df['annual_inc'] > 70000)  & (df['annual_inc'] <= 80000), 1, 0)
df['annual_inc:80K-90K']   = np.where((df['annual_inc'] > 80000)  & (df['annual_inc'] <= 90000), 1, 0)
df['annual_inc:90K-100K']  = np.where((df['annual_inc'] > 90000)  & (df['annual_inc'] <= 100000), 1, 0)
df['annual_inc:100K-120K'] = np.where((df['annual_inc'] > 100000) & (df['annual_inc'] <= 120000), 1, 0)
df['annual_inc:120K-140K'] = np.where((df['annual_inc'] > 120000) & (df['annual_inc'] <= 140000), 1, 0)
df['annual_inc:>140K']     = np.where((df['annual_inc'] > 140000), 1, 0)

### Months since last delinquency

In [197]:
df['mths_since_last_delinq:Missing'] = np.where((df['mths_since_last_delinq'].isnull()), 1, 0)
df['mths_since_last_delinq:0-3'] = np.where((df['mths_since_last_delinq'] >= 0) & (df['mths_since_last_delinq'] <= 3), 1, 0)
df['mths_since_last_delinq:4-30'] = np.where((df['mths_since_last_delinq'] >= 4) & (df['mths_since_last_delinq'] <= 30), 1, 0)
df['mths_since_last_delinq:31-56'] = np.where((df['mths_since_last_delinq'] >= 31) & (df['mths_since_last_delinq'] <= 56), 1, 0)
df['mths_since_last_delinq:>=57'] = np.where((df['mths_since_last_delinq'] >= 57), 1, 0)  

### Debt to income 

In [198]:
df['dti:<=1.4']     = np.where((df['dti'] <= 1.4), 1, 0)
df['dti:1.4-3.5']   = np.where((df['dti'] > 1.4)  & (df['dti'] <= 3.5), 1, 0)
df['dti:3.5-7.7']   = np.where((df['dti'] > 3.5)  & (df['dti'] <= 7.7), 1, 0)
df['dti:7.7-10.5']  = np.where((df['dti'] > 7.7)  & (df['dti'] <= 10.5), 1, 0)
df['dti:10.5-16.1'] = np.where((df['dti'] > 10.5) & (df['dti'] <= 16.1), 1, 0)
df['dti:16.1-20.3'] = np.where((df['dti'] > 16.1) & (df['dti'] <= 20.3), 1, 0)
df['dti:20.3-21.7'] = np.where((df['dti'] > 20.3) & (df['dti'] <= 21.7), 1, 0)
df['dti:21.7-22.4'] = np.where((df['dti'] > 21.7) & (df['dti'] <= 22.4), 1, 0)
df['dti:22.4-35']   = np.where((df['dti'] > 22.4) & (df['dti'] <= 35), 1, 0)
df['dti:>35']       = np.where((df['dti'] > 35), 1, 0)

### Months since last record

In [199]:
df['mths_since_last_rec:Missing'] = np.where((df['mths_since_last_record'].isnull()), 1, 0)
df['mths_since_last_rec:0-2']     = np.where((df['mths_since_last_record'] >= 0)  & (df['mths_since_last_record'] <= 2), 1, 0)
df['mths_since_last_rec:3-20']    = np.where((df['mths_since_last_record'] >= 3)  & (df['mths_since_last_record'] <= 20), 1, 0)
df['mths_since_last_rec:21-31']   = np.where((df['mths_since_last_record'] >= 21) & (df['mths_since_last_record'] <= 31), 1, 0)
df['mths_since_last_rec:32-48']   = np.where((df['mths_since_last_record'] >= 32) & (df['mths_since_last_record'] <= 48), 1, 0)
df['mths_since_last_rec:49-60']   = np.where((df['mths_since_last_record'] >= 49) & (df['mths_since_last_record'] <= 60), 1, 0)
df['mths_since_last_rec:61-84']   = np.where((df['mths_since_last_record'] >= 61) & (df['mths_since_last_record'] <= 84), 1, 0)
df['mths_since_last_rec:84-99']   = np.where((df['mths_since_last_record'] >= 84) & (df['mths_since_last_record'] <= 99), 1, 0)
df['mths_since_last_rec:>100']    = np.where((df['mths_since_last_record'] > 100), 1, 0)

In [165]:
# main = df

In [200]:
second = df

In [201]:
main.shape

(415445, 327)

In [202]:
second.shape

(257073, 327)

In [203]:
main.columns.values == second.columns.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [204]:
main.to_csv("/content/drive/MyDrive/Credit Risk Modeling/train.csv")

In [205]:
second.to_csv("/content/drive/MyDrive/Credit Risk Modeling/test.csv")

In [212]:
# main.columns.values
necessary_columns = [
'grade:A', 'grade:B', 'grade:C', 'grade:D',
'grade:E', 'grade:F', 'grade:G',
###
'home_ownership:ANY', 'home_ownership:MORTGAGE',
'home_ownership:NONE', 'home_ownership:OTHER',
'home_ownership:OWN', 'home_ownership:RENT',
###
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
###
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
###
'initial_list_status:f',
'initial_list_status:w',
### 
'addr_state:NE_IA_NV_AL_NM',
'addr_state:HI_MO_VA', 'addr_state:OK_TN_NJ_NC_KY',
'addr_state:MD_UT_LA_MN_MI',
'addr_state:SD_WA_AZ_OH_PA_MA_AR_DE_IN_GA', 'addr_state:WI_RI_OR',
'addr_state:CT_IL', 'addr_state:MT_KS_CO_SC_AK',
'addr_state:VT_MS_NH_WV_WY_DC_ME_ID',
'addr_state:FL',
'addr_state:CY',
'addr_state:NY',
'addr_state:TX',
###
'emp_length:1', 'emp_length:2-4', 'emp_length:5-6',
'emp_length:7-9', 'emp_length:10', 
###
'since_issue_date:<87',
'since_issue_date:<90', 'since_issue_date:<92',
'since_issue_date:<94', 'since_issue_date:<99',
'since_issue_date:<103', 'since_issue_date:<112',
'since_issue_date:<142', 'since_issue_date:>=142',
###
'interest_rate:<9.548', 'interest_rate:9.548-12.025',
'interest_rate:12.025-14.089', 'interest_rate:14.089-15.74',
'interest_rate:15.74-20.281', 'interest_rate:>20.281',
###
'earl_cr_line:<209', 'earl_cr_line:209-285',
'earl_cr_line:285-323', 'earl_cr_line:323-380',
'earl_cr_line:380-437', 'earl_cr_line:>=437', 
###
'delinq_2yrs:0',
'delinq_2yrs:1-3', 'delinq_2yrs:4-11', 'delinq_2yrs:>=12',
###
'inq_last_6mths:0', 'inq_last_6mths:1-2', 'inq_last_6mths:3-6',
'inq_last_6mths:>6', 
###
'open_acc:0', 'open_acc:1-3', 'open_acc:4-9',
'open_acc:10-12', 'open_acc:13-17', 'open_acc:18-22',
'open_acc:23-27', 'open_acc:28-35', 'open_acc:>=36', 
###
'pub_rec:0',
'pub_rec:1-2', 'pub_rec:3-5', 'pub_rec:>=6', 
###
'total_acc:<=3',
'total_acc:4-15', 'total_acc:16-21', 'total_acc:22-27',
'total_acc:28-48', 'total_acc:49-63', 'total_acc:64-78',
'total_acc:>=79', 
###
'acc_now_delinq:0', 'acc_now_delinq:>=1',
###
'total_rev_hi_lim:<=5K', 'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-15K', 'total_rev_hi_lim:15K-20K',
'total_rev_hi_lim:20K-30K', 'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K', 'total_rev_hi_lim:55K-95K',
'total_rev_hi_lim:>95K', 
###
'annual_inc:<20K', 'annual_inc:20K-30K',
'annual_inc:30K-40K', 'annual_inc:40K-50K', 'annual_inc:50K-60K',
'annual_inc:60K-70K', 'annual_inc:70K-80K', 'annual_inc:80K-90K',
'annual_inc:90K-100K', 'annual_inc:100K-120K',
'annual_inc:120K-140K', 'annual_inc:>140K',
###
'mths_since_last_delinq:Missing', 'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30', 'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57', 
###
'dti:<=1.4', 'dti:1.4-3.5',
'dti:3.5-7.7', 'dti:7.7-10.5', 'dti:10.5-16.1', 'dti:16.1-20.3',
'dti:20.3-21.7', 'dti:21.7-22.4', 'dti:22.4-35', 'dti:>35',
###
'mths_since_last_rec:Missing', 'mths_since_last_rec:0-2',
'mths_since_last_rec:3-20', 'mths_since_last_rec:21-31',
'mths_since_last_rec:32-48', 'mths_since_last_rec:49-60',
'mths_since_last_rec:61-84', 'mths_since_last_rec:84-99',
'mths_since_last_rec:>100', 
###
'good_loan']


In [208]:
len(main.columns)

327

In [211]:
len(necessary_columns)

137